In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
import pytesseract
import time
import os

# Đường dẫn đến Tesseract OCR (cập nhật đúng với vị trí cài đặt)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Cấu hình Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(options=options)

# Mở trang đăng nhập
url = "https://dichvucong.moit.gov.vn/Login.aspx"
driver.get(url)

# Chờ các phần tử cần thiết hiển thị
wait = WebDriverWait(driver, 10)

try:
    # Nhập username
    username_field = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="ctl00_cplhContainer_txtLoginName"]')))
    username_field.send_keys("2300956022")  # Thay bằng username thực tế của bạn

    # Nhập password
    password_field = driver.find_element(By.XPATH, '//*[@id="ctl00_cplhContainer_txtPassword"]')
    password_field.send_keys("2300956022")  # Thay bằng password thực tế của bạn

    # Lấy ảnh captcha
    captcha_image = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="ctl00_cplhContainer_imgCaptcha"]')))
    captcha_src = captcha_image.get_attribute("src")
    print(f"Captcha URL: {captcha_src}")

    # Tải ảnh captcha
    captcha_path = "captcha.png"
    driver.save_screenshot("screenshot.png")  # Chụp toàn bộ màn hình
    location = captcha_image.location
    size = captcha_image.size

    # Crop ảnh captcha từ ảnh màn hình
    x = location['x']
    y = location['y']
    width = location['x'] + size['width']
    height = location['y'] + size['height']

    image = Image.open("screenshot.png")
    captcha = image.crop((x, y, width, height))
    captcha.save(captcha_path)
    print("Captcha đã được lưu.")

    # Đọc mã captcha bằng Tesseract OCR
    captcha_code = pytesseract.image_to_string(captcha, config='--psm 8').strip()
    print(f"Mã captcha OCR: {captcha_code}")

    # Nhập mã captcha
    captcha_field = driver.find_element(By.XPATH, '//*[@id="ctl00_cplhContainer_txtCaptcha"]')
    captcha_field.send_keys(captcha_code)

    # Nhấn nút đăng nhập
    login_button = driver.find_element(By.XPATH, '//*[@id="ctl00_cplhContainer_btnLogin"]')
    login_button.click()

    # Chờ tải xong sau khi đăng nhập
    time.sleep(5)

    # Kiểm tra đăng nhập thành công hay không
    if "Dashboard" in driver.page_source:
        print("Đăng nhập thành công!")
    else:
        print("Đăng nhập thất bại. Kiểm tra lại captcha hoặc thông tin đăng nhập.")
finally:
    # Đóng trình duyệt
    # driver.quit()

    # Xóa file ảnh đã lưu
    if os.path.exists("screenshot.png"):
        os.remove("screenshot.png")
    if os.path.exists("captcha.png"):
        os.remove("captcha.png")


Captcha URL: https://dichvucong.moit.gov.vn/GeneralCaptchaHandler.ashx?iwidth=120&iheight=50&t=638701340580216007
Captcha đã được lưu.


TesseractNotFoundError: C:\Program Files\Tesseract-OCR\tesseract.exe is not installed or it's not in your PATH. See README file for more information.